In [18]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch
import pandas as pd
from PIL import ImageDraw, ImageFont, Image
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
import torch.nn as nn
from transformers import ViTModel
from torchinfo import summary  # 
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import random
import time

In [19]:
DEVICE="cuda:1"
def setAllSeeds(seed):
  os.environ['MY_GLOBAL_SEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
setAllSeeds(42)

In [20]:
df = pd.read_csv("train.csv")
categories=df["Category"].unique()
print(categories)
category=categories[1]
df = df[df["Category"]==category]

['Men Tshirts' 'Sarees' 'Kurtis' 'Women Tshirts' 'Women Tops & Tunics']


In [21]:
delCol = []
idxCol = []
trackNum = []
for i in range(1,11):
    uniName = df["attr_"+str(i)].unique()
    # print(len(uniName))
    if(len(uniName)==1):
        delCol.append("attr_"+str(i))
    else:
        idxCol.append("attr_"+str(i))
        trackNum.append(len(uniName))

In [22]:
df = df.drop(delCol,axis=1)
df.shape

(18346, 13)

In [23]:
df.isna().sum()

id              0
Category        0
len             0
attr_1      10461
attr_2        667
attr_3       2485
attr_4        450
attr_5        697
attr_6      13336
attr_7       9450
attr_8       1881
attr_9       4043
attr_10       528
dtype: int64

In [24]:
id2label={}
label2id={}
attrs={}
total_attr=len(df.columns)
for i in range(3,total_attr):
    labels=df[df.columns[i]].dropna().unique()
    # print(df.columns[i],labels)
    id2label[i-3]={k:labels[k] for k in range(len(labels))}
    label2id[i-3]={labels[k]:k for k in range(len(labels))}
    attrs[i-3]=df.columns[i]
print(id2label)
print(label2id)
print(attrs)

{0: {0: 'same as saree', 1: 'solid', 2: 'same as border', 3: 'default'}, 1: {0: 'woven design', 1: 'zari', 2: 'no border', 3: 'solid', 4: 'default', 5: 'temple border'}, 2: {0: 'small border', 1: 'big border', 2: 'no border'}, 3: {0: 'multicolor', 1: 'cream', 2: 'white', 3: 'default', 4: 'navy blue', 5: 'yellow', 6: 'green', 7: 'pink'}, 4: {0: 'party', 1: 'traditional', 2: 'daily', 3: 'wedding'}, 5: {0: 'jacquard', 1: 'default', 2: 'tassels and latkans'}, 6: {0: 'woven design', 1: 'same as saree', 2: 'default', 3: 'zari woven'}, 7: {0: 'zari woven', 1: 'woven design', 2: 'default', 3: 'solid', 4: 'printed'}, 8: {0: 'applique', 1: 'elephant', 2: 'floral', 3: 'ethnic motif', 4: 'peacock', 5: 'default', 6: 'solid', 7: 'checked', 8: 'botanical'}, 9: {0: 'no', 1: 'yes'}}
{0: {'same as saree': 0, 'solid': 1, 'same as border': 2, 'default': 3}, 1: {'woven design': 0, 'zari': 1, 'no border': 2, 'solid': 3, 'default': 4, 'temple border': 5}, 2: {'small border': 0, 'big border': 1, 'no border': 

In [25]:
def categorize(example):
    for i in attrs:
        # print(example[attrs[i]],type(example[attrs[i]]),pd.isna(example[attrs[i]]))
        if not pd.isna(example[attrs[i]]):
            example[attrs[i]]=label2id[i][example[attrs[i]]]
        else:
            example[attrs[i]]=-100
    return example
df=df.apply(categorize,axis=1)
df.head()

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
7267,7432,Sarees,10,0,0,0,0,0,0,0,0,0,0
7268,7433,Sarees,10,-100,1,0,1,1,-100,-100,0,1,0
7269,7434,Sarees,10,-100,1,0,2,0,-100,-100,0,2,0
7270,7435,Sarees,10,0,0,1,3,1,-100,1,0,3,0
7271,7436,Sarees,10,1,2,-100,-100,2,-100,-100,-100,-100,1


In [26]:
df.isna().sum()

id          0
Category    0
len         0
attr_1      0
attr_2      0
attr_3      0
attr_4      0
attr_5      0
attr_6      0
attr_7      0
attr_8      0
attr_9      0
attr_10     0
dtype: int64

In [27]:
from transformers import AutoImageProcessor
# model_name = 'google/vit-base-patch16-224'
model_name="facebook/deit-base-distilled-patch16-224"
processor = AutoImageProcessor.from_pretrained(model_name)

In [28]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.3)
val_df,test_df=train_test_split(val_df,test_size=0.33)

In [29]:
class CustomFashionManager(Dataset):
    def __init__(self,csv_file, root_dir="./",transforms =None):
        self.fashionItems = csv_file
        self.root_dir = root_dir
        self.transforms = transforms
    
    def __len__(self):
        return len(self.fashionItems)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,f"{self.fashionItems.iloc[idx, 0]:06d}"+'.jpg')
        image = Image.open(img_name)
        attributes = self.fashionItems.iloc[idx, 3:]
        attributes = np.array(attributes)
        attributes = attributes.astype('float')
        # print(attributes.shape)
        # attributes = attributes.astype('float').reshape(-1, len(attributes))
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        inputs=processor(image, return_tensors='pt')
        inputs['labels']=torch.tensor(attributes, dtype=torch.long)
        return inputs

        # if self.transforms:
        #     sample = self.transforms(sample)

        # return sample


In [30]:
train_fashion_data = CustomFashionManager(csv_file=train_df,
                                    root_dir='train_images')
val_fashion_data = CustomFashionManager(csv_file=val_df,
                                    root_dir='train_images')
test_fashion_data = CustomFashionManager(csv_file=test_df,root_dir='train_images')

fig = plt.figure()
        

<Figure size 640x480 with 0 Axes>

In [31]:
import sys
from typing import List
from transformers import ViTConfig,ViTPreTrainedModel,DeiTConfig,DeiTPreTrainedModel,DeiTModel


class CustomConfig(DeiTConfig):
    def __init__(self,num_classes_per_label:List[int]=[1],**kwargs):
        super().__init__(**kwargs)
        self.num_classes_per_label = num_classes_per_label

class MultiLabelMultiClassViT(DeiTPreTrainedModel):
    config_class=CustomConfig
    def __init__(self, config: CustomConfig) -> None:
        super().__init__(config)

        self.vit = DeiTModel(config, add_pooling_layer=False)
        # for param in self.vit.parameters():
        #     param.requires_grad = False
        self.classifiers = nn.ModuleList([
            nn.Linear(config.hidden_size, num_classes) 
            for num_classes in config.num_classes_per_label
        ])
        # Initialize weights and apply final processing
        self.post_init()


    def forward(self, pixel_values,labels=None):
        outputs = self.vit(pixel_values).last_hidden_state[:, 0, :]  # CLS token representation
        logits = [classifier(outputs) for classifier in self.classifiers]
        if labels is not None:
            loss=0
            for i in range(len(logits)):
                target=labels[:,i]
                loss += torch.nn.functional.cross_entropy(logits[i], target)
            return {"loss": loss, "logits": logits}
        return {"logits": logits}

# Example usage
num_labels = len(trackNum)  # For example, 5 different labels



In [32]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import classification_report
batch_size = 32
def collate_fn(batch):
    return {
        'pixel_values': torch.cat([x['pixel_values'] for x in batch],dim=0),
        'labels': torch.stack([x['labels'] for x in batch])
    }

def compute_metrics(pred):
    logits = pred.predictions
    labels=pred.label_ids
    probs = np.stack([np.argmax(logit,axis=1) for logit in logits])
    probs=probs.T
    # truth_labels=[]
    # preds=[]
    # for i in range(len(probs)):
    #     pred=[]
    #     true=[]
    #     for j in range(len(probs[i])):
    #         pred.append(id2label[j][probs[i][j]])
    #         if labels[i][j]==-100:
    #             true.append(-100)
    #         else:
    #             true.append(id2label[j][labels[i][j]])
    #     preds.append(pred)
    #     truth_labels.append(true)

    # preds=np.array(preds)
    # truth_labels=np.array(truth_labels)

    # labels=truth_labels.flatten()
    # probs=preds.flatten()

    labels=labels.flatten()
    probs=probs.flatten()
    non_padding_indices = [i for i, label in enumerate(labels) if label != '-100']

# Use the filtered indices to get non-padding true and predicted labels
    labels = [labels[i] for i in non_padding_indices]
    probs = [probs[i] for i in non_padding_indices]

    # print(classification_report(labels,probs))
    report=classification_report(labels,probs,output_dict=True)
    return {'accuracy': report['accuracy'],"macro avg f1":report['macro avg']['f1-score']}

training_args = TrainingArguments(
  output_dir="./vit3/"+category,
  per_device_train_batch_size=64,
  per_device_eval_batch_size=64,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  logging_strategy="epoch",
  num_train_epochs=5,
  fp16=True,
  learning_rate=3e-4,
  save_total_limit=1,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  load_best_model_at_end=True,
  metric_for_best_model="macro avg f1"
)
config=ViTConfig.from_pretrained(model_name)
config=CustomConfig(num_classes_per_label=trackNum,**config.to_dict())
model = MultiLabelMultiClassViT.from_pretrained(model_name,config=config)

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_fashion_data,
    eval_dataset=val_fashion_data,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

You are using a model of type deit to instantiate a model of type vit. This is not supported for all configurations of models and can yield errors.
Some weights of MultiLabelMultiClassViT were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['deit.classifiers.0.bias', 'deit.classifiers.0.weight', 'deit.classifiers.1.bias', 'deit.classifiers.1.weight', 'deit.classifiers.2.bias', 'deit.classifiers.2.weight', 'deit.classifiers.3.bias', 'deit.classifiers.3.weight', 'deit.classifiers.4.bias', 'deit.classifiers.4.weight', 'deit.classifiers.5.bias', 'deit.classifiers.5.weight', 'deit.classifiers.6.bias', 'deit.classifiers.6.weight', 'deit.classifiers.7.bias', 'deit.classifiers.7.weight', 'deit.classifiers.8.bias', 'deit.classifiers.8.weight', 'deit.classifiers.9.bias', 'deit.classifiers.9.weight', 'deit.vit.embeddings.cls_token', 'deit.vit.embeddings.distillation_token', 'deit.vit.embeddings.patch_embeddings.projection.bias', 'd

In [ ]:
trainer.train()
trainer.save_model(f"./vit3/{category}/final")

Epoch,Training Loss,Validation Loss,Accuracy,Macro avg f1
1,8.429700,7.394188,0.531326,0.354912
2,6.952000,6.751800,0.545755,0.409123


In [17]:
trainer.evaluate(test_fashion_data)

KeyboardInterrupt: 